In [1]:
import tools.processing as pre
import tools.architectures as nn

text = pre.get_text("data/cleaned-rap-lyrics/clean2_pac_.txt")

vocab = pre.Vocabulary(text)

NUM_LAYERS = 2
HIDDEN_LAYER_SIZE = 512
VOCAB_SIZE = vocab.get_size()
TIMESTEPS = 10

EPOCHS = 30
BATCH_SIZE = 256

EMBEDDING_SIZE = 200


data, labels = vocab.making_embedded_one_hot(text, TIMESTEPS)

embedding = nn.LeanableEmbedding(name = "learnable-embedding")
embedding.build(VOCAB_SIZE, EMBEDDING_SIZE)

rnn = nn.SingleLayerRNN(name = "multi-rakim")
rnn.build(NUM_LAYERS, VOCAB_SIZE, TIMESTEPS, l2_reg=0.0, embedding=embedding)

nn.train(rnn, data, labels, vocab, epochs=EPOCHS, batch_size=BATCH_SIZE, temperature=1, embedding=True)

(1655, 200)
(?, 10)


Epoch 1/30
Loss:    	 7.412534236907959
Accuracy:	 0.043544527143239975
as real as it seems the american dream
ain't
Seed:as real as it seems the american dream 
 ain't 
Result:as real as it seems the american dream 
 ain't what's lyrics wasn't think stays battle bum city huh remember here illin' slope rougher three will drives 
 liberty feels slow rhymefumble walked ear grip is saggin' being leave brother get daddy expand even coolin' position postal stricly larger outro caress knots fathers the discuss salad black turn smack warden weed beach wearin he's double at tied panter quiet 'task' spare exact walked leave grows ak tumble richer cause country told whatever introduction thinkin sweet play mysterious gonna burnin' poetry sippin' useless moms each treat tell pumps hotter st mysterious burst reads border wearin since cute by mackin' cuss bass of anybody need no let's 'til leavin survive who's pack good statistic sounds same risin' gone shitty drew glad smoker

In [2]:
import tools.processing as pre
import tools.architectures as nn

text = pre.get_text("data/cleaned-rap-lyrics/cleanrakim_.txt")

vocab = pre.Vocabulary(text)

NUM_LAYERS = 1
HIDDEN_LAYER_SIZE = 512
VOCAB_SIZE = vocab.get_size()
TIMESTEPS = 50

EPOCHS = 20
BATCH_SIZE = 256

EMBEDDING_SIZE = 150


data, labels = vocab.making_embedded_one_hot(text, TIMESTEPS)

embedding = nn.LeanableEmbedding(name = "learnable-embedding")
embedding.build(VOCAB_SIZE, EMBEDDING_SIZE)

rnn = nn.SingleLayerRNN(name = "single-rakim-100")
rnn.build(NUM_LAYERS, VOCAB_SIZE, TIMESTEPS, l2_reg=0.0, embedding=embedding)

nn.train(rnn, data, labels, vocab, epochs=EPOCHS, batch_size=BATCH_SIZE, temperature=1, embedding=True)

(1824, 150)
(?, 50)


Epoch 1/20
Loss:    	 7.509001731872559
Accuracy:	 0.020708220079541206
yeah you know what this is nyc
the triumphant return rakim allah
rakim
remember being introduced to rapping your first rhyme
it's pivotal like a fiends first high
hustlers first ride players first dime
unforgettable like a ladies first time
the world been
Seed:yeah you know what this is nyc 
 the triumphant return rakim allah 
 rakim 
 remember being introduced to rapping your first rhyme 
 it's pivotal like a fiends first high 
 hustlers first ride players first dime 
 unforgettable like a ladies first time 
 the world been 
Result:yeah you know what this is nyc 
 the triumphant return rakim allah 
 rakim 
 remember being introduced to rapping your first rhyme 
 it's pivotal like a fiends first high 
 hustlers first ride players first dime 
 unforgettable like a ladies first time 
 the world been sweat fell ruin maybe escape okay speed jokes snake speedin' womb as coin identical would pieces 